In [2]:
import pandas as pd
import sqlite3
import datetime as dt
import altair as alt
from pandasql import sqldf
pysqldf=lambda q :sqldf(q,globals())

import plotly as py
import plotly.express as px
import plotly.io as pio
pio.renderers.default='browser'
import json


In [3]:
conn=sqlite3.connect('CovidIndia.sqlite')
Data=pd.read_sql_query("SELECT *  FROM Data",conn)
State=pd.read_sql_query("SELECT *  FROM State",conn)
State_Data=pd.read_sql_query("SELECT *  FROM State_Data",conn)
#df=pd.read_sql_query("SELECT Date,Confirmed, Recovered, Deceased, Tested,Vaccinated, State_Name, State_Area,State_Population  FROM Data LEFT JOIN State_Data ON Data.state_id=State_Data.state_id",conn)
#df.to_excel("Test.xlsx")
#Date,Confirmed, Recovered, Deceased, Tested,Vaccinated ,State_Population 
#df=pd.read_sql_table(conn,conn)

State_Data.head()

,state_id,state_code,State_Name,State_Area,State_Population
0,19207,UP,Uttar Pradesh,243286.0,240250000
1,10968,MH,Maharashtra,307713.0,124184000
2,2343,BR,Bihar,94163.0,110020000
3,20410,WB,West Bengal,88752.0,96320000
4,12724,MP,Madhya Pradesh,308252.0,85344000


In [4]:
#x=lambda a,b:a+b
#x(1,2)


In [5]:
Joined_DF=pysqldf("Select state_code, Date, Confirmed,Recovered,Deceased,Tested,Vaccinated1,Vaccinated2,State_Name,State_Area,State_Population from Data left join State on Data.state_id=State.id left join State_Data on Data.state_id=State_Data.state_id")
Joined_DF.head()


,state_code,Date,Confirmed,Recovered,Deceased,Tested,Vaccinated1,Vaccinated2,State_Name,State_Area,State_Population
0,AN,2020-03-26,1,0,0,0,0,0,Andaman and Nicobar Islands,8073.0,380520
1,AN,2020-03-27,5,0,0,0,0,0,Andaman and Nicobar Islands,8073.0,380520
2,AN,2020-03-28,3,0,0,0,0,0,Andaman and Nicobar Islands,8073.0,380520
3,AN,2020-03-29,0,0,0,0,0,0,Andaman and Nicobar Islands,8073.0,380520
4,AN,2020-03-30,1,0,0,0,0,0,Andaman and Nicobar Islands,8073.0,380520


In [6]:
Joined_DF["Date"]=pd.DatetimeIndex(Joined_DF['Date'])
Joined_DF.dtypes

state_code                  object
Date                datetime64[ns]
Confirmed                    int64
Recovered                    int64
Deceased                     int64
Tested                       int64
Vaccinated1                  int64
Vaccinated2                  int64
State_Name                  object
State_Area                 float64
State_Population             int64
dtype: object

In [7]:
alt.data_transformers.enable('json') #To avoid MaxRowsError

select_timeline=alt.selection_interval(encodings=["x"])  # as slider
bar_slider=alt.Chart(Joined_DF).mark_bar().encode(
    y="sum(Confirmed):Q",
    x="Date:T"
    ).add_selection(select_timeline
                   ).properties(title="Bar Slider",height=50)  #now bar slider will act like a slider

Conf_line=alt.Chart(Joined_DF).mark_line().encode(
    x="Date:T",
    y="sum(Confirmed):Q",
    tooltip="sum(Confirmed):Q",
    color=alt.condition(select_timeline,alt.value("black"),alt.value("lightgray"))
).properties(title="Detailed - Confirmed")   #defining basic chart for confirmed cases

Death_line=alt.Chart(Joined_DF).mark_line().encode(
    x="Date:T",
    y="sum(Deceased):Q",
    tooltip="sum(Deceased):Q",
    color=alt.condition(select_timeline,alt.value("black"),alt.value("lightgray"))
) .properties(title="Detailed - Deaths")     #defining basic chart for death cases

States=pysqldf("select distinct(State_Name) from Joined_DF")["State_Name"].values.tolist() #making a list of distinct states
states_dropdown=alt.binding_select(options=States) 
states_select=alt.selection_single(fields=["State_Name"],bind=states_dropdown,name="Select")  #dropdown for states

filter_conf=Conf_line.add_selection(
    states_select
).transform_filter(
    states_select
).transform_filter(
    select_timeline)  #applying necessary transform_filter

filter_death=Death_line.add_selection(
    states_select
).transform_filter(
    states_select
).transform_filter(
    select_timeline
)



In [8]:
#Defining cumulative charts for the country overall
Conf_cum=alt.Chart(Joined_DF).mark_line().encode(
    x="Date:T",
    y="sum(Confirmed):Q",
    tooltip="sum(Confirmed):Q"
).properties(title="National Count Confirmed").interactive()   #defining basic chart for confirmed cases

Death_cum=alt.Chart(Joined_DF).mark_line().encode(
    x="Date:T",
    y="sum(Deceased):Q",
    tooltip="sum(Deceased):Q"
).properties(title="National count Deaths").interactive()    #defining basic chart for death cases

Conf_cum | Death_cum

alt.HConcatChart(...)

In [9]:
filter_conf | filter_death & bar_slider 

alt.HConcatChart(...)

In [10]:
Table_Bubble=alt.Chart(Joined_DF).mark_circle().encode(
    y="state_code:N",
    x="Date:T",
    size="sum(Confirmed)").transform_filter(
    select_timeline)

Table_Bubble & bar_slider 

alt.VConcatChart(...)

In [11]:
Summary_Table=pysqldf("Select State_Name,sum(Confirmed) as 'Total cases', sum(Deceased) as 'Total Deaths', sum(Vaccinated1) as 'Partially_Vax', sum(Vaccinated2) as 'Fully_Vax'"
           "from Joined_DF group by state_code order by sum(confirmed) DESC")
Summary_Table

#https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html - use this to stylize the data


,State_Name,Total cases,Total Deaths,Partially_Vax,Fully_Vax
0,Maharashtra,6611078,140216,67198794,30975692
1,Kerala,4968657,31681,25306499,13658343
2,Karnataka,2988333,38082,42497761,22858384
3,Tamil Nadu,2702623,36116,41279432,17619141
4,Andhra Pradesh,2066450,14373,32976969,20375181
5,Uttar Pradesh,1710158,22900,98178865,32681895
6,West Bengal,1592908,19141,56192166,21559747
7,Delhi,1439870,25091,13055636,7425404
8,Odisha,1041457,8386,25736641,11560912
9,Chhattisgarh,1006052,13577,14851682,7343273


In [12]:
Joined_DF["State_Name"].replace({"Andaman and Nicobar Islands":"Andaman & Nicobar Island","Arunachal Pradesh":"Arunanchal Pradesh",
                               "Daman and Diu":"Daman & Diu","Delhi":"NCT of Delhi","Jammu and Kashmir":"Jammu & Kashmir",
                               "Pondicherry":"Puducherry"},inplace=True)
Joined_DF

,state_code,Date,Confirmed,Recovered,Deceased,Tested,Vaccinated1,Vaccinated2,State_Name,State_Area,State_Population
0,AN,2020-03-26,1,0,0,0,0,0,Andaman & Nicobar Island,8073.0,380520
1,AN,2020-03-27,5,0,0,0,0,0,Andaman & Nicobar Island,8073.0,380520
2,AN,2020-03-28,3,0,0,0,0,0,Andaman & Nicobar Island,8073.0,380520
3,AN,2020-03-29,0,0,0,0,0,0,Andaman & Nicobar Island,8073.0,380520
4,AN,2020-03-30,1,0,0,0,0,0,Andaman & Nicobar Island,8073.0,380520
...,...,...,...,...,...,...,...,...,...,...,...
20998,WB,2021-10-27,976,837,15,43322,323292,229056,West Bengal,88752.0,96320000
20999,WB,2021-10-28,990,845,9,45437,532317,287998,West Bengal,88752.0,96320000
21000,WB,2021-10-29,982,860,8,49017,701727,317576,West Bengal,88752.0,96320000
21001,WB,2021-10-30,980,880,13,47131,756507,304433,West Bengal,88752.0,96320000


In [13]:
# list1=[]
# i=0
# while i<=35:
#     list1.append(geojson_data["features"][i]["properties"]["st_nm"])
#     i=i+1
    
# tempdf=pysqldf("Select distinct State_Name from Joined_DF")
# tempdf["check"]=sorted(list1)

In [14]:
# Map of India with data

geojson_data=pd.read_json("states_india.geojson")

#geojson_data["features"][1]["properties"] format of geojson

state_id_dict={}
for feature in geojson_data["features"]:
    state_id_dict[feature["properties"]["st_nm"]]=feature["properties"]["state_code"]
state_id_dict["Ladakh"]=99 #since Ladakh is not in our geojson file

Joined_DF['id']=Joined_DF["State_Name"].apply(lambda x:state_id_dict[x])


In [14]:
# geojson_data = json.load(open("states_india.geojson", "r"))

# Summary_Table["State_Name"].replace({"Andaman and Nicobar Islands":"Andaman & Nicobar Island","Arunachal Pradesh":"Arunanchal Pradesh",
#                                "Daman and Diu":"Daman & Diu","Delhi":"NCT of Delhi","Jammu and Kashmir":"Jammu & Kashmir",
#                                "Pondicherry":"Puducherry"},inplace=True)
# Summary_Table['id']=Summary_Table["State_Name"].apply(lambda x:state_id_dict[x])


In [19]:
Summary_Table["State_Name"].replace({"Andaman and Nicobar Islands":"Andaman & Nicobar Island","Arunachal Pradesh":"Arunanchal Pradesh",
                               "Daman and Diu":"Daman & Diu","Delhi":"NCT of Delhi","Jammu and Kashmir":"Jammu & Kashmir",
                               "Pondicherry":"Puducherry"},inplace=True)
Summary_Table['id']=Summary_Table["State_Name"].apply(lambda x:state_id_dict[x])
Summary_Table1=Summary_Table.to_json()
Joined_DF1=Joined_DF.to_json()
Summary_Table.drop(index=32,inplace=True)


KeyError: '[32] not found in axis'

In [20]:
Summary_Table=Summary_Table.astype({"id":int})
Summary_Table.dtypes

State_Name       object
Total cases       int64
Total Deaths      int64
Partially_Vax     int64
Fully_Vax         int64
id                int32
dtype: object

In [17]:
# Summary_Table["Total Deaths"].plot()

In [21]:
fig=px.choropleth(
                Summary_Table,
                locations='id',
                geojson=geojson_data.to_json(),
                color="Total Deaths",
                hover_name="State_Name",
                hover_data=["Total Deaths"],
                title="Sample Map"
             )
fig.update_geos(fitbounds="locations",visible=False)
fig.show()

In [19]:
#Exploratory on top 10 contributors, multi colored bar chart, tree map etc.

In [20]:
#Visuals stating total vaccination doses, vax 1 as % of population, vax 2 as % of population, testing per 100 people